# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import requests
import os
from bs4 import BeautifulSoup
from PIL import Image
import time

# single thread ---------------------------------------------------------------
strtTime = time.time()

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

output_dir = 'day035'
if not os.path.exists(output_dir):
         os.makedirs(output_dir)
         
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
for img_tag in image_tags:
    if 'imgur' not in img_tag['href']:
        continue
    img_id = img_tag['href'].split('/')[-1]
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    with requests.get(img_url, stream=True) as r:
        r.raise_for_status()
        img = Image.open(r.raw)
        imgFmt=img.format.lower()
        if (imgFmt!='jpeg'):
            continue
        img_savename = '{outdir}/{img_id}.{imgFmt}'.format(
            outdir=output_dir, img_id=img_id, imgFmt=imgFmt)
        img.save(img_savename)
        print('Save {} file: {}'.format(imgFmt, img_savename))

endTime = time.time()
print(endTime - startTime)

# multi  thread ---------------------------------------------------------------
import _thread

def imgurl_req(imgurl, id):
    with requests.get(imgurl, stream=True) as r:
        r.raise_for_status()
        img = Image.open(r.raw)
        imgFmt=img.format.lower()
        if (imgFmt=='jpeg'):
            img_savename = '{outdir}/{img_id}.{imgFmt}'.format(
                outdir=output_dir, img_id=id, imgFmt=imgFmt)
            img.save(img_savename)
            print('Save {} file: {}'.format(imgFmt, img_savename))

strtTime = time.time()

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)

output_dir = 'day035'
if not os.path.exists(output_dir):
         os.makedirs(output_dir)
 
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
for img_tag in image_tags:
    if 'imgur' not in img_tag['href']:
        continue
    img_id = img_tag['href'].split('/')[-1]
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    _thread.start_new_thread( imgurl_req, (img_url, img_id) )

endTime = time.time()
print(endTime - startTime)

'\nYour Code\n'

In [ ]:
single thread: 1272.8367352485657